In [10]:
import cython
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pymc3 as pm

from utils.car_model import CAR2

from theano import scan
import theano.tensor as tt

from pymc3.distributions import continuous
from pymc3.distributions import distribution

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-darkgrid')

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train/255
x_test  = x_test/255

dummy = x_train[12]

def subset(label, data=x_train, labels=y_train):
    '''
    e.g. subset(3) -> gets all pictures of digit 3
    '''
    assert label >= 0 and label <= 9
    
    return data[np.argwhere(labels == 3)]

In [5]:
O = np.concatenate(dummy)

adj = []
position_matrix = np.linspace(0, 28*28 - 1, num=28*28).astype(np.int64).reshape(28, 28)
count = 0

for i, row in enumerate(position_matrix):
    for j, col in enumerate(position_matrix[i]):
        assert position_matrix[i][j] == col
        
        temp = []

        # change these loops if we do not want to
        # include diagonal elements in adj matrix
        for delta_i in [-1, 0, 1]:
            for delta_j in [-1, 0, 1]:
                if ((i + delta_i) // 28 == 0) and ((j + delta_j) // 28 == 0):    
                    temp.append(position_matrix[i + delta_i][j + delta_j])
        

        temp.remove(col)
        temp.sort()
        adj.append(temp)
        
weights = [list(np.ones_like(adj_elems).astype(np.int64)) for adj_elems in adj]

# below is taken from the pymc3 CAR tutorial website
maxwz = max([sum(w) for w in weights])
N = len(weights)
wmat2 = np.zeros((N, N))
amat2 = np.zeros((N, N), dtype='int32')
for i, a in enumerate(adj):
    amat2[i, a] = 1
    wmat2[i, a] = weights[i]

In [11]:
def expit(x):
    return tt.exp(x)/(1 + tt.exp(x))

with pm.Model() as model:
    beta0  = pm.Normal('beta0', mu=0., tau=1e-2)
    tau    = pm.Gamma('tau_c', alpha=1.0, beta=1.0)
    mu_phi = CAR2('mu_phi', w=wmat2, a=amat2, tau=tau, shape=N)
    phi    = pm.Deterministic('phi', mu_phi-tt.mean(mu_phi)) # zero-center phi
    
    pi_0 = pm.Uniform('pi_0', lower=0, upper=1)
    pi_1 = pm.Uniform('pi_1', lower=0, upper=1-pi_0)
    
    mu = pm.Deterministic('mu', beta0 + phi)
    logit_Yi = pm.LogitNormal('Yi', mu=mu, observed=pad(O))
    Yi = expit(logit_yi)
    Zi = pm.Mixture('Zi', w=[pi_0, pi_1, 1 - pi_0 - pi_1], )
    
    trace = pm.sample(draws=2000, cores=1)
    posterior_pred = pm.sample_posterior_predictive(trace)

NameError: name 'pad' is not defined

In [ ]:
pm.Mixture()